In [3]:
import pandas as pd

# Load the raw file
# --- This sample data is the correct form as it downloaded from the software as it contain many unwanted rows and columns.
file_path = 'Sample_data.csv'
# --- Skiprows = 3 is given as the first 3 rows contain just the headings
df = pd.read_csv(file_path, skiprows=3,dtype={'Phone2': str})
# --- Only wanted columns are taken 
df = df[['Bill Date', 'OUTLET NAME', 'Customer Name', 'Phone2']]
# ---  Skipping the first row after column name as it is a total of of orher columns
df = df.iloc[1:].copy()

df['Phone2'] = df['Phone2'].astype(str).str.replace(r'\D+', '', regex=True).str.strip()

# Function for conditions
def classify(row):
    # making every customer name capital to follow same pattern
    name = str(row['Customer Name']).strip().upper()
    phone = str(row['Phone2'])
    # Classifing customer care number, A customer name 'customer_care' only have number 9091600900
    if name == 'CUSTOMER_CARE' or phone == '9091600900':
        return 'Customer Care'
    # if the row is blank it classifies to 'Number Not Provided'
    elif phone == '' or pd.isna(phone):
        return 'Number Not Provided'
    # If there is fake number is given
    elif len(phone) != 10 or not phone.isdigit():
        return 'Not a Phone Number'
    else:
        return 'Actual Customer'

df['Category'] = df.apply(classify, axis=1)

# Group and summarize
summary = df.groupby(['Bill Date', 'OUTLET NAME']).agg(
    NOB=('Phone2', 'count'),
    Customercare_Num=('Category', lambda x: (x == 'Customer Care').sum()),
    Actual_Cus=('Category', lambda x: (x == 'Actual Customer').sum()),
    Not_Provided=('Category', lambda x: (x == 'Number Not Provided').sum()),
    Not_a_Phone=('Category', lambda x: (x == 'Not a Phone Number').sum())
).reset_index()

# Rename and calculate %
summary.rename(columns={
    'Customercare_Num': 'Customercare Num',
    'Actual_Cus': 'Actual Cus',
    'Not_Provided': 'Number Not Provided',
    'Not_a_Phone': 'Not a Phone Number'
}, inplace=True)

summary['Customercare %'] = summary['Customercare Num'] / summary['NOB']
summary['Actual Cus %'] = summary['Actual Cus'] / summary['NOB']
summary['Number Not Provided %'] = summary['Number Not Provided'] / summary['NOB']

# Add Grand Total row
grand_total = {
    'Bill Date': '',
    'OUTLET NAME': 'Grand Total',
    'NOB': summary['NOB'].sum(),
    'Customercare Num': summary['Customercare Num'].sum(),
    'Actual Cus': summary['Actual Cus'].sum(),
    'Number Not Provided': summary['Number Not Provided'].sum(),
    'Not a Phone Number': summary['Not a Phone Number'].sum()
}
grand_total['Customercare %'] = grand_total['Customercare Num'] / grand_total['NOB']
grand_total['Actual Cus %'] = grand_total['Actual Cus'] / grand_total['NOB']
grand_total['Number Not Provided %'] = grand_total['Number Not Provided'] / grand_total['NOB']

summary = pd.concat([summary, pd.DataFrame([grand_total])], ignore_index=True)

# Write to Excel with chart and formatting
output_file = 'daily_phone_summary.xlsx'
with pd.ExcelWriter(output_file, engine='xlsxwriter') as writer:
    summary.to_excel(writer, sheet_name='Summary', index=False)
    workbook = writer.book
    worksheet = writer.sheets['Summary']

    # Create chart (exclude Grand Total)
    row_count = len(summary) - 1
    chart = workbook.add_chart({'type': 'column'})

    for col_name, color in [
        ('Actual Cus %', 'green'),
        ('Customercare %', 'blue'),
        ('Number Not Provided %', 'red')
    ]:
        col_idx = summary.columns.get_loc(col_name)
        chart.add_series({
            'name':       ['Summary', 0, col_idx],
            'categories': ['Summary', 1, 1, row_count, 1],
            'values':     ['Summary', 1, col_idx, row_count, col_idx],
            'fill':       {'color': color},
        })

    chart.set_title({'name': 'Phone Classification % by Outlet'})
    chart.set_x_axis({'name': 'Outlet'})
    chart.set_y_axis({'name': 'Percentage', 'major_gridlines': {'visible': False}})
    chart.set_style(10)
    worksheet.insert_chart('L2', chart)

    # Apply percentage formatting
    percent_format = workbook.add_format({'num_format': '0.00%'})
    for col_name in ['Customercare %', 'Actual Cus %', 'Number Not Provided %']:
        col_idx = summary.columns.get_loc(col_name)
        worksheet.set_column(col_idx, col_idx, 18, percent_format)

        # Apply color scale conditional formatting
        worksheet.conditional_format(1, col_idx, row_count, col_idx, {
            'type': '3_color_scale',
            'min_color': "#F8696B",
            'mid_color': "#FFEB84",
            'max_color': "#63BE7B"
        })

print(f"✅ Summary of Phone number Classification: {output_file}")


✅ Summary of Phone number Classification: daily_phone_summary.xlsx
